In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#dt = pd.read_csv("/kaggle/input/dialogue-lines-of-the-simpsons/simpsons_dataset.csv")
dt = pd.read_csv("/kaggle/input/anime-subtitles/input (Cleaned).txt", sep='\n', header=None)
dt.columns = ["spoken_words"]

In [ ]:
dt = dt.dropna()
dt

In [ ]:
dt['spoken_words'][0]

# Preprocessing
In this step we need to clean the sentences, there are a lot of steps that we can apply, we need to remove all characters that are not text from the sentences, then we can remove the stop words, the words that have pretty much no meaning and also we can 

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()
# using english stopwords
stopword_list=nltk.corpus.stopwords.words('english')

def remove_stopwords(text): # remove stopwords, words with no usefull meaning
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def remove_special_characters(text, remove_digits=True): # removing special characters useless in this context
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

def simple_stemmer(text): # apply stemmer to reach the root of some words for better generalization
    ps = nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
dt['spoken_words'] = dt['spoken_words'].apply(remove_special_characters)
dt['spoken_words'] = dt['spoken_words'].apply(remove_stopwords)
dt['spoken_words'] = dt['spoken_words'].apply(simple_stemmer)

In [ ]:
dt['spoken_words'][0]

In [ ]:
#clean_dt = pd.DataFrame({'clean': txt})
dt = dt.dropna()
dt = dt.drop_duplicates()
dt.shape
dt

# Create model

In [ ]:
from gensim.models import Word2Vec
import multiprocessing
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in dt['spoken_words']]
phrases = Phrases(sent)
bigram = Phraser(phrases)
sentences = bigram[sent]

In [ ]:
cores = multiprocessing.cpu_count()
bigrams_model = Word2Vec(
    min_count=20,
    alpha=0.005,
    workers=cores-1)
bigrams_model.build_vocab(sentences, progress_per=10000)

In [ ]:
bigrams_model.train(sentences, total_examples=bigrams_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
print("Vocabulary size:", len(bigrams_model.wv.vocab))

# Testing
we can now ask the model to give us similarities or finding the word that does not belong to a list

In [ ]:
bigrams_model.wv.most_similar('bullet')

In [ ]:
bigrams_model.wv.similarity("magic", "spell")

In [ ]:
bigrams_model.wv.most_similar('natsu')

In [ ]:
bigrams_model.wv.doesnt_match(['natsu', 'erza', 'gildart', 'goku'])

Since Goku is from a different series from the rest the answer is correct.